In [27]:
from typing import Dict, List, Tuple, Optional
import networkx as nx
import numpy as np
import cvxpy as cp
import os, sys
import time
import torch
import random

# if __name__ == "__main__":
#     current_dir = os.path.dirname(os.path.abspath(__file__))
#     parent_dir = os.path.dirname(current_dir)
#     sys.path.append(parent_dir)

from utils.logging_utils import LogManager
from utils.partition_utils import ScheduleConstPartitionSolver
from utils.task_graph_generation import TaskGraphDataset, create_data_lists

def create_test_instances():
    """
    Creates a test DAG with the following structure:
    
         1
        / \
       2   3
       |   |
       4   5
           |
           6
    
    Nodes: 6
    Edges: 5 -> (1,2), (1,3), (2,4), (3,5), (5,6)

    HW_costs: 4, 3, 1, 1, 2, 3
    HW_areas: 10, 10, 10, 3, 3, 3
    SW_costs: 5, 4, 1, 10, 20, 30
    All edges have communication cost of 1
    Max_HW_Use = 10
    
    """
    G = nx.DiGraph()

    # Add nodes explicitly
    nodes = [0, 1, 2, 3, 4, 5]
    G.add_nodes_from(nodes)

    # Add edges
    edges = [(0, 1), (0, 2), (1, 3), (2, 4), (4, 5)]
    G.add_edges_from(edges)

    sw_costs = torch.Tensor([5,4,1,10,20,30])
    hw_costs = torch.Tensor([4,3,1,1,2,3])
    hw_areas = torch.Tensor([10,10,10,3,3,3])

    comm_costs = torch.Tensor([1 for i in range(len(edges))])
    
    return G,hw_costs,sw_costs,hw_areas,comm_costs

def create_test_instances2():
    """
    Creates a test DAG with the following structure:
    
         1
        / | \
       2  7   3
       |  |  |
       4  8   5
             |
             6
    
    Nodes: 8
    Edges: 5 -> (1,2), (1,3), (2,4), (3,5), (5,6), (1,7), (7,8)

    HW_costs: 4, 3, 1, 3, 7, 8, 2, 3
    HW_areas: 10, 10, 10, 3, 3, 3, 10, 10
    SW_costs: 5, 4, 1, 10, 20, 30, 2, 3
    All edges have communication cost of 1
    Max_HW_Use = 10, not relevant for forced test
    
    """
    G = nx.DiGraph()

    # Add nodes explicitly
    nodes = [0, 1, 2, 3, 4, 5, 6, 7]
    G.add_nodes_from(nodes)

    # Add edges
    edges = [(0, 1), (0, 2), (1, 3), (2, 4), (4, 5), (0,6), (6,7)]
    G.add_edges_from(edges)

    sw_costs = torch.Tensor([5, 4, 1, 10, 20, 30, 2, 3])
    hw_costs = torch.Tensor([4, 3, 1, 3, 7, 8, 2, 3])
    hw_areas = torch.Tensor([10, 10, 10, 3, 3, 3, 10, 10])

    comm_costs = torch.Tensor([1 for i in range(len(edges))])
    
    return G,hw_costs,sw_costs,hw_areas,comm_costs

## Forced Partitioning and Scheduling Test

### We force nodes 1, 2, 7, 3, 8 in software and 4, 5, 6 in software.

After executing node 1, we have a choice regarding 2,7 and 3 -- we need to schedule 3 first, scheduling 7 or 2 should give higher makespan 

In [28]:
g,hw_cost,sw_cost,hw_area,comm_cost = create_test_instances2()
partition = {0:0,1:0,2:0,3:1,4:1,5:1,6:0,7:0}

solver = ScheduleConstPartitionSolver()
solver.load_networkx_graph_with_torch_feats(g, hw_area, hw_cost, sw_cost, comm_cost)
sol = solver.solve_optimization(A_max=10.0, partition_assignment=partition)
print(f"MILP solution: makepan={sol['makespan']}, start times = {sol['start_times']}")

(CVXPY) Feb 17 10:42:09 AM: Your problem has 96 variables, 179 constraints, and 0 parameters.
(CVXPY) Feb 17 10:42:09 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 17 10:42:09 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 17 10:42:09 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 17 10:42:09 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Feb 17 10:42:09 AM: Compiling problem (target solver=SCIP).
(CVXPY) Feb 17 10:42:09 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) Feb 17 10:42:09 AM: Applying reduction Dcp2Cone
(CVXPY) Feb 17 10:42:09 AM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 17 10:42:09 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Feb 17 10:42:09 AM: Applying reduction SCIP
(CVXPY) Feb 17 10:42:09 AM: Finished problem compilation (took 1

                                     CVXPY                                     
                                     v1.7.2                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
presolving:
(round 1, fast)       72 del vars, 120 del conss, 0 add conss, 56 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 2, fast)       77 del vars, 125 del conss, 0 add conss, 70 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 0 clqs
(round 3, exhaustive) 77 del vars, 156 del conss, 0 add conss, 70 chg bounds, 

We can see that the MIP has scheduled 2(3-1) first and 6(7) next and 1(2) next

## Joined partitioning and scheduling

We expect the makespan to be at least as low as 22 here, since we are not enforcing any partition here

In [29]:
solver = ScheduleConstPartitionSolver()
solver.load_networkx_graph_with_torch_feats(g, hw_area, hw_cost, sw_cost, comm_cost)
solution = solver.solve_optimization(A_max=10.0)
print(f"MILP solution: makepan={solution['makespan']}, start times = {solution['start_times']}")


(CVXPY) Feb 17 10:43:19 AM: Your problem has 96 variables, 164 constraints, and 0 parameters.
(CVXPY) Feb 17 10:43:19 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Feb 17 10:43:19 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Feb 17 10:43:19 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Feb 17 10:43:19 AM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Feb 17 10:43:19 AM: Compiling problem (target solver=SCIP).
(CVXPY) Feb 17 10:43:19 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> SCIP
(CVXPY) Feb 17 10:43:19 AM: Applying reduction Dcp2Cone
(CVXPY) Feb 17 10:43:19 AM: Applying reduction CvxAttr2Constr
(CVXPY) Feb 17 10:43:19 AM: Applying reduction ConeMatrixStuffing
(CVXPY) Feb 17 10:43:19 AM: Applying reduction SCIP
(CVXPY) Feb 17 10:43:19 AM: Finished problem compilation (took 9

                                     CVXPY                                     
                                     v1.7.2                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
presolving:
(round 1, fast)       39 del vars, 20 del conss, 0 add conss, 22 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 59 clqs
(round 2, fast)       39 del vars, 20 del conss, 0 add conss, 35 chg bounds, 0 chg sides, 0 chg coeffs, 0 upgd conss, 0 impls, 59 clqs
(round 3, fast)       39 del vars, 20 del conss, 0 add conss, 43 chg bounds, 0